# NYC data & social distancing fits (SIHR)

This notebook compares the SIHR model fits of the NYC data with or without implemented social distancing measures. 
It is an exact copy of the SIR equivalent with the difference that now the `sihr_step` was used. The conlcusion don't change.

**TL;DR:** 

1. The data is described best if a social distancing measures are in place.
2. Precise knowledge about new admissions is more important than knowledge about new infections assuming a 15% relative uncertainty in new admissions and a 50% or larger uncertainty in new known cases.
3. To reliably run social distancing fits, a "bend" in the data (deviation from exponential growth) should be visible.
4. Once the bend is visible, predictions are consistent and fit precision seems limited by data. Predictions in the range of a week seem feasible, after that, the extrapolation uncertainty becomes large.

# Init

## Imports

In [1]:
from os import environ
from datetime import date, timedelta

from pandas import DataFrame, date_range, Series
from numpy import array, linspace, log, sqrt, exp, arange

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gvar import gvar, mean, sdev
from lsqfit import nonlinear_fit

from plotting import COLUMN_NAME_MAP, plot_fits, summarize_fit, plot_fit_range
from models import sihr_step, one_minus_logistic_fcn, FitFcn
from prepare_df import prepare_case_hosp_death, COMMIT_HASH_LAST

print(COMMIT_HASH_LAST)

e19db289166f73282d39dfcef0d47a324d654c07


## Data

In [2]:
BIN_SIZE = 2
# See also the NYC-data-preparation notebook for choice analysis

chd_df = prepare_case_hosp_death(
    COMMIT_HASH_LAST,  # Specify NYC repo commit hash to ensure same data
    bin_day_range=BIN_SIZE,  # How many days should be grouped as one
    drop_days_end=3,  # Drop rows where date awas 3 days within reporting
).loc[
    "2020-03-08":"2020-04-01"
]  # Only consider given time range
chd_df.tail()

,death_cummulative,death_new,hospitalized_cummulative,hospitalized_new,infected_cummulative,infected_new
date,,,,,,
2020-03-24,476,199,7254,2136,33350,7932
2020-03-26,822,346,9702,2448,41731,8381
2020-03-28,1299,477,11984,2282,47538,5807
2020-03-30,1884,585,14594,2610,56423,8885
2020-04-01,2582,698,16964,2370,65525,9102


# Fitting

# Prepare fit

To fit the SIR model, I distinguish between two sets of parameters:

1. Fit data stored in a `XX` and `YY` variable. This data does not change after being specified once.
2. Fit parameters stored in a prior which specifies the initial belive. These parameters are optimized while fitting.

In [3]:
## This is data considerd fix in fit
XX = {
    "initial_susceptible": int(8.6e6),  # https://en.wikipedia.org/wiki/New_York_City
    "initial_hospitalized": chd_df.hospitalized_cummulative.iloc[0],
    "initial_recovered": 0,
    "n_iter": chd_df.shape[0] + 1,  # Because of the bin size, one iteration = 3 days
    "bin_size": BIN_SIZE,  # To convert units later on
    "capacity": 20000
}
print(XX)

{'initial_susceptible': 8600000, 'initial_hospitalized': 102, 'initial_recovered': 0, 'n_iter': 14, 'bin_size': 2, 'capacity': 20000}


For this fit to work, the `YY` variable must also quantify uncertainty.
I believe that daily new admissions to a hospital (`hospitalized_new`) are more accurate than a count of daily new infections (`infected_new`, I have no estimation about how accurate the estimation of new infections is at all). Thus I emphasize `hospitalized_new` over `infected_new`.
In this context, the $\chi^2$ per d.o.f. is not necessarily meaningful and uncertainties rather quantify relative weights if how much you trust individual data points. 

In [4]:
infected_new = chd_df["infected_new"].values
hospitalized_new = chd_df["hospitalized_new"].values

# This assumes that there is a 50% uncertainty in the number of infected people
## And at least 300 (if the number is small to not emphasize early measurments too much)
delta_infected_new = [max(300, infected * 0.5) for infected in infected_new]

# This assumes that there is a 10% uncertainty in the number of hospitalized people with a minimum of 50
delta_hospitalized_new = [hospitalized * 0.15 for hospitalized in hospitalized_new]

YY = gvar(
    [infected_new, hospitalized_new], [delta_infected_new, delta_hospitalized_new]
).T

# Fits

**About doubling time and social distancing**

If one assumes, in the initial phase of the COVID-19 spread, $S(t) \approx S_0$, one has

$$
    \dot I(t) = (\beta_I S_0 - \gamma_I) I(t)
    \quad \Rightarrow \quad
    I(t) = I_0 \exp\{(\beta_I S_0 - \gamma_I)t\}
$$

Thus, the initial doubling time is defined as

$$
    I(t_2) = 2 I_0 = I_0 \exp\{(\beta_I S_0 - \gamma_I)t_2\} 
    \quad \Rightarrow \quad 
    t_2 = \frac{\ln(2)}{\beta_I S_0 - \gamma_I}
    \quad \Rightarrow \quad 
    \beta_I =  \frac{1}{S_0} \left[ \frac{\ln(2)}{t_2} +  \gamma_I \right]
$$

In later fits, social distancing policies are introduced. 
Thus, the growth rate $\beta_I$ becomes a function of time which is implemented by a logistic function with three unknown coefficients

$$
    \beta_I(t; R, t_0, \Delta t) = \beta_I \left[ 1 - \frac{R}{1+\exp\{-(t-t_0)\Delta t\}}\right]
$$



## SIHR, no social distancing policies

In [12]:
# Describe the fit function
fcn = FitFcn(
    sihr_step,  # run regular SIR model
    columns=[
        "infected_new",
        "hospitalized_new",
    ],  # return only data for these two columns
    as_array=True,  # return array not df
    drop_rows=[0],  # drop first row (since new values are NaN in this row)
)

# And the prior estimates of fit parameters
## gvars are gaussian random numbers described by their mean and standard deviation
prior = {
    # 5(2) days is the estimated initial doubling time
    "inital_doubling_time": gvar(5, 2),
    # Days until one is recovered
    ## Assumption: same for infected only or in hospital
    "recovery_days_i": gvar(14, 5),
    "recovery_days_h": gvar(14, 5),
    # Wild guess of how many where initially infected
    "initial_infected": gvar(1.0e4, 2.0e4),
    # Rate of infected people becoming hospitalized
    "beta_h": gvar(0.1, 0.5),
}

# Run the fit
fit = nonlinear_fit(data=(XX, YY), fcn=fcn, prior=prior)

# And present it
summarize_fit(fit)
fig = plot_fits(fit, x=chd_df.index[1:])
fig.show()

Legend:
- I(0): initial_infected
- t2(0): inital_doubling_time
- Recovery I: recovery_days_i
- Recovery H: recovery_days_h 

Least Square Fit:
  chi2/dof [dof] = 5.5 [26]    Q = 3.8e-18    logGBF = -267.46

Parameters:
inital_doubling_time    2.44 (22)     [     5.0 (2.0) ]  *
     recovery_days_i   14.2 (5.0)     [    14.0 (5.0) ]  
     recovery_days_h   14.0 (5.0)     [    14.0 (5.0) ]  
    initial_infected    407 (112)     [ 10000 (20000) ]  
              beta_h   0.179 (47)     [     0.10 (50) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 21/0.1)
  fitter = scipy_least_squares    method = trf
 
------- 
Result: 
------- 

 Values:
             beta_h: 0.179(47)           
               I(0): 407(112)            
              t2(0): 2.44(22)            
         Recovery I: 14.2(5.0)           
         Recovery H: 14.0(5.0)           
 
 
------------- 
Error budget: 
------------- 

 Partial Errors:
                beta_h       I(0)      t2

## SIHR with social distancing policies

In [6]:
# Fit function now with beta_i_fcn
fcn_w_distancing = FitFcn(
    sihr_step,
    columns=["infected_new", "hospitalized_new"],
    beta_i_fcn=one_minus_logistic_fcn,
    as_array=True,
    drop_rows=[0],
)

# Copy prior from before but add social distancing parameters
prior_w_distancing = prior.copy()
# Maximal reduction of social distancing: R
prior_w_distancing["ratio"] = gvar(0.7, 0.3)
# How many days to go from ~ r to 0.5r: Delta t
prior_w_distancing["social_distance_halfing_days"] = gvar(14, 7)
# After how many days the measures hits 0.5r: t0
prior_w_distancing["social_distance_delay"] = gvar(14, 7)

# Run the fit
fit_w_distancing = nonlinear_fit(
    data=(XX, YY), fcn=fcn_w_distancing, prior=prior_w_distancing
)

# Summarize the fit
summarize_fit(fit_w_distancing)
fig = plot_fits(fit_w_distancing, x=chd_df.index[1:])
fig.show()

Legend:
- I(0): initial_infected
- t2(0): inital_doubling_time
- Recovery I: recovery_days_i
- Recovery H: recovery_days_h
- R: ratio
- w sd: social_distance_halfing_days
- dt sd: social_distance_delay 

Least Square Fit:
  chi2/dof [dof] = 0.67 [26]    Q = 0.89    logGBF = -210.61

Parameters:
        inital_doubling_time    1.02 (34)     [     5.0 (2.0) ]  *
             recovery_days_i    9.2 (4.4)     [    14.0 (5.0) ]  
             recovery_days_h   14.0 (5.0)     [    14.0 (5.0) ]  
            initial_infected     235 (74)     [ 10000 (20000) ]  
                      beta_h   0.202 (58)     [     0.10 (50) ]  
                       ratio    0.76 (15)     [     0.70 (30) ]  
social_distance_halfing_days    4.2 (2.7)     [    14.0 (7.0) ]  *
       social_distance_delay    8.3 (3.3)     [    14.0 (7.0) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 46/1.6)
  fitter = scipy_least_squares    method = trf
 
------- 
Result: 
------- 

 Values:
  

## SIHR with social distancing policies & larger infected uncertainty

It is likely that the number of known cases (infections) is significantly underestimated.
How does the fit change if we blow up the uncertainty of `infected_new` by 300% of it's mean value?

In [13]:
infected_new = chd_df["infected_new"].values
hospitalized_new = chd_df["hospitalized_new"].values

# This assumes that there is a 300% uncertainty in the number of infected people
## And at least 300 (if the number is small to not emphasize early measurments too much)
delta_infected_new = [max(300, infected * 3) for infected in infected_new]

# This assumes that there is a 10% uncertainty in the number of hospitalized people with a minimum of 50
delta_hospitalized_new = [hospitalized * 0.15 for hospitalized in hospitalized_new]

YY_larger_I = gvar(
    [infected_new, hospitalized_new], [delta_infected_new, delta_hospitalized_new]
).T

In [14]:
# Run the fit
fit_w_distancing_larger_I = nonlinear_fit(
    data=(XX, YY_larger_I), fcn=fcn_w_distancing, prior=prior_w_distancing
)

# Summarize the fit
summarize_fit(fit_w_distancing_larger_I)
fig = plot_fits(fit_w_distancing_larger_I, x=chd_df.index[1:])
fig.show()

Legend:
- I(0): initial_infected
- t2(0): inital_doubling_time
- Recovery I: recovery_days_i
- Recovery H: recovery_days_h
- R: ratio
- w sd: social_distance_halfing_days
- dt sd: social_distance_delay 

Least Square Fit:
  chi2/dof [dof] = 0.41 [26]    Q = 1    logGBF = -225.92

Parameters:
        inital_doubling_time    0.85 (38)     [     5.0 (2.0) ]  **
             recovery_days_i   13.8 (5.0)     [    14.0 (5.0) ]  
             recovery_days_h   14.0 (5.0)     [    14.0 (5.0) ]  
            initial_infected    150 (135)     [ 10000 (20000) ]  
                      beta_h    0.32 (28)     [     0.10 (50) ]  
                       ratio    0.80 (19)     [     0.70 (30) ]  
social_distance_halfing_days    5.3 (3.4)     [    14.0 (7.0) ]  *
       social_distance_delay    7.5 (4.3)     [    14.0 (7.0) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 59/0.5)
  fitter = scipy_least_squares    method = trf
 
------- 
Result: 
------- 

 Values:
    

In [15]:
DataFrame(
    data=[
        fit_w_distancing.p,
        fit_w_distancing_larger_I.p,
        fit_w_distancing.p - fit_w_distancing_larger_I.p,
    ],
    index=["50% infected new uncertainty", "300% infected new uncertainty", "diff"],
).T

,50% infected new uncertainty,300% infected new uncertainty,diff
beta_h,0.202(58),0.32(28),-0.12(28)
inital_doubling_time,1.02(34),0.85(38),0.17(37)
initial_infected,235(74),150(135),85(145)
ratio,0.76(15),0.80(19),-0.04(17)
recovery_days_h,14.0(5.0),14.0(5.0),0.0(1.6)e-13
recovery_days_i,9.2(4.4),13.8(5.0),-4.6(2.4)
social_distance_delay,8.3(3.3),7.5(4.3),0.7(3.9)
social_distance_halfing_days,4.2(2.7),5.3(3.4),-1.1(2.9)


Results are mostly consistend and thus, new admissions dictate the outcome of the fit. Note that there is an almost one-sigma discrepancy in the rate of infected people becoming hospitalized.

## Predictivity check: Regular SIR, with social distancing policies

The final question is: how far can we predict? Or alternatively, if we leave out time slices, how many do we need to consistently describe the data?

In [11]:
NT = YY.shape[0]

fits = {}

for nt in range(NT // 3, NT + 1):
    yy_cut = YY[:nt].copy()
    xx = XX.copy()
    xx["n_iter"] = nt + 1
    fits[nt] = nonlinear_fit(
        data=(xx, yy_cut), fcn=fcn_w_distancing, prior=prior_w_distancing
    )

fig = plot_fit_range(fits, y_max=5000, x=chd_df.index[1:], col_wrap=3)

fig.show()
DataFrame(
    data=[fit.p for fit in fits.values()],
    index=Series([nt * XX["bin_size"] for nt in fits], name="fitted days"),
)

,beta_h,inital_doubling_time,initial_infected,ratio,recovery_days_h,recovery_days_i,social_distance_delay,social_distance_halfing_days
fitted days,,,,,,,,
8,0.31(14),0.85(20),151(72),0.69(30),14.0(5.0),14.1(5.0),14.4(7.0),13.3(7.0)
10,0.28(10),0.89(19),169(68),0.68(30),14.0(5.0),14.1(5.0),14.5(7.0),13.4(7.0)
12,0.236(79),0.95(21),206(73),0.73(29),14.0(5.0),14.0(5.0),13.8(6.9),12.4(6.9)
14,0.227(74),0.94(39),211(73),0.87(27),14.0(5.0),13.3(5.0),9.0(5.7),5.8(5.1)
16,0.220(68),0.96(39),215(72),0.85(27),14.0(5.0),13.0(5.0),8.4(5.0),4.9(3.9)
18,0.210(61),0.97(38),227(71),0.81(26),14.0(5.0),12.9(5.0),8.0(5.1),5.1(3.7)
20,0.204(56),1.01(37),233(71),0.72(23),14.0(5.0),12.8(4.9),7.3(4.1),4.1(3.3)
22,0.203(57),0.99(36),235(72),0.81(21),14.0(5.0),11.6(4.8),8.3(3.8),4.8(3.4)
24,0.199(54),1.04(34),238(72),0.75(17),14.0(5.0),11.3(4.7),8.0(3.2),3.9(2.8)


Conclusion: Fits seem to be consistent once you see a "bend" in the new hospitalizations curve.
E.g., after fitted days $\geq 14$, the social distancing function parameters are affected by the fit.